In [16]:
from batchcreator import DataGenerator, get_list_IDs
from datetime import datetime
import tensorflow as tf
from model_builder import build_generator

In [19]:
# Create list of IDs to retrieve
x_seq_size=5
y_seq_size=1

start_dt = datetime(2019,6,1,0,0)
end_dt =  datetime(2019,6,30,0)
# Exclude samples were there is no rain in the input
filter_no_rain = False
list_IDs = get_list_IDs(start_dt, end_dt, x_seq_size, y_seq_size, filter_no_rain=filter_no_rain)
print(list_IDs[0])
print(len(list_IDs))

(['201905312335', '201905312340', '201905312345', '201905312350', '201905312355'], ['201906010000'])
1671


In [24]:
generator = DataGenerator(list_IDs, batch_size=256, x_seq_size=x_seq_size, 
                                       y_seq_size=y_seq_size, normalize_y=False)

In [25]:
for i in generator:
    x,y = i
    print(x.shape, y.shape)

(256, 5, 768, 700, 1) (256, 1, 384, 350, 1)
(256, 5, 768, 700, 1) (256, 1, 384, 350, 1)
(256, 5, 768, 700, 1) (256, 1, 384, 350, 1)
(256, 5, 768, 700, 1) (256, 1, 384, 350, 1)
(256, 5, 768, 700, 1) (256, 1, 384, 350, 1)
(256, 5, 768, 700, 1) (256, 1, 384, 350, 1)


In [17]:
model = build_generator()
model.summary()
#plot_model(model)
model.compile( optimizer='sgd',
    loss='mse',
    metrics=[tf.keras.metrics.MeanSquaredError()])

Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 5, 768, 700, 1)]  0         
_________________________________________________________________
Downsample1a (Conv3D)        (None, 5, 384, 350, 8)    80        
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 5, 384, 350, 8)    0         
_________________________________________________________________
Downsample1b (Conv2D)        (None, 5, 192, 175, 8)    584       
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 5, 192, 175, 8)    0         
_________________________________________________________________
ConvLSTM1a (ConvLSTM2D)      (None, 5, 192, 175, 64)   166144    
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 5, 192, 175, 64)   0 

In [18]:
hist = model.fit(generator, epochs=3)

Epoch 1/3
2/2 [==============================] - 69s 24s/step - loss: 794.3126 - mean_squared_error: 794.3126
Epoch 2/3
2/2 [==============================] - 48s 24s/step - loss: 1986.5670 - mean_squared_error: 1986.5671
Epoch 3/3
2/2 [==============================] - 48s 24s/step - loss: 780.2461 - mean_squared_error: 780.2461
